In [21]:
import numpy as np
import pandas as pd
import requests

# Football Match Prediction

### Introduction
The goal of this project is to develop a model for predicting the outcomes of football matches using mathematical techniques, specifically linear algebra and matrix operations. By analyzing various factors such as team statistics, player performance, and historical match results, the model aims to estimate the most likely outcome (win, loss, or draw) of a football match between two teams.


In [43]:
# Function for getting team id by name for using it 
def get_team_id_by_name(team_name, api_key):
    url = f"https://v3.football.api-sports.io/teams?search={team_name}"

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "v3.football.api-sports.io"
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        if data["results"] > 0:
            team_id = data["response"][0]["team"]["id"]
            return team_id
        else:
            return None  # Team not found
    else:
        print(f"Error: {response.status_code}")
        return None

# Usage example:
# api_key = "f27a22f13747e86db3d869c26b306756"  # Replace with your API key
# team_name1 = "Levski Sofia"
# team_name2 = "Real Madrid"
# team_id1 = get_team_id_by_name(team_name1, api_key)
# team_id2 = get_team_id_by_name(team_name2, api_key)
# print(team_id1, team_id2)



In [51]:
# Function to get the last 5 matches of a team
def get_last_5_matches(api_key, team_id):
    url = f"https://v3.football.api-sports.io/teams?search={team_name}"

    # Headers for the request
    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "v3.football.api-sports.io"
    }

    # Send request to API
    response = requests.get(f'{url}?team={team_id}', headers=headers)

    if response.status_code == 200:
        # Get the JSON response
        matches = response.json()['response']
        
        # Sort matches by date (from most recent to oldest)
        matches.sort(key=lambda x: x['fixture']['date'], reverse=True)
        
        # Get the last 5 matches
        last_5_matches = matches[:5]
        
        # Display the last 5 matches
        for match in last_5_matches:
            home_team = match['teams']['home']['name']
            away_team = match['teams']['away']['name']
            date = match['fixture']['date']
            # Determine which team is the home team or away team and get the score
            if match['teams']['home']['id'] == team_id:
                score = f"{match['goals']['home']} - {match['goals']['away']}"
            else:
                score = f"{match['goals']['away']} - {match['goals']['home']}"
            
            print(f"{home_team} vs {away_team} - Date: {date} - Score: {score}")
    else:
        print(f"Error fetching data: {response.status_code}")

# Usage example:
api_key = "f27a22f13747e86db3d869c26b306756"  # Replace with your API key
team_name1 = "Levski Sofia"
team_name2 = "Real Madrid"
team_id1 = get_team_id_by_name(team_name1, api_key)
team_id2 = get_team_id_by_name(team_name2, api_key)
print(get_last_5_matches(api_key, team_id2))


None


In [11]:
# Функция за получаване на данни за даден отбор
def get_team_data(team_name):
    url = f"https://api.football-data.org/v2/teams/{team_name}"  # API URL за даден отбор
    headers = {"X-Auth-Token": "YOUR_API_KEY"}  # Заменете с вашия API ключ

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        team_data = response.json()
        # Примерни данни, които можеш да вземеш от отговора на API-то
        # Тук взимаме атака и защита (но може да варира в зависимост от структурата на данните)
        attack = team_data['squad'][0]['attack']  # Примерен ключ
        defense = team_data['squad'][0]['defense']  # Примерен ключ
        return attack, defense
    else:
        return None


In [13]:
# Функция за вземане на мачовете между два отбора
def get_head_to_head_matches(team1_name, team2_name):
    # Заявка за извличане на данни за мачовете между двата отбора
    url = f"https://api.football-data.org/v2/matches"
    headers = {"X-Auth-Token": "YOUR_API_KEY"}  # Заменете с вашия API ключ

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        match_data = response.json()
        
        # Филтриране на мачовете между двата отбора
        matches = []
        for match in match_data['matches']:
            if (match['homeTeam']['name'] == team1_name and match['awayTeam']['name'] == team2_name) or \
               (match['homeTeam']['name'] == team2_name and match['awayTeam']['name'] == team1_name):
                matches.append(match)
        
        return matches
    else:
        print("Error fetching data:", response.status_code)
        return None